<a href="https://colab.research.google.com/github/venkat52/my_colab_notes/blob/main/Module1_Lab2_FMML20210326.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>
 Module Coordinator: Thrupthi Ann John thrupthi.ann@research.iiit.ac.in <br>
 Release date: 11 October 2021 Monday <br>

 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In [ ]:
p=rng.random(10)
p>0.9

array([False, False, False, False, False,  True, False, False, False,
       False])

In [ ]:
a=np.array([1,2,3,4]).reshape(2,2)
a.sum(1)

array([3, 7])

In [ ]:
k=datasets.fetch_california_housing()
k.feature_names
# k.target

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [ ]:
a=[[1,2,3,4,5,16],[1,2,3,4,5,6],[9,3,4,5,5,1]]
b=[1,2,3,4,5,6]
# if a==b:
#   print('hello')
s=([True,False,True])
print(type(s))
a[1:]

<class 'list'>


[[1, 2, 3, 4, 5, 6], [9, 3, 4, 5, 5, 1]]

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. 

Let us download and examine the dataset. 

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(sum(dataset.data[0]))
 print(dataset.target)

297.538692063492
[4 3 3 ... 0 0 0]


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  # print(traindata)
  # print(query,'qurey')
  # return 0
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  # print(sq)
  dist = sq.sum(1) # add up the squares
  # print(dist.shape)
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel


We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  # print(classes)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  # print(rints)
  predlabel = classes[rints]
  # print(predlabel)
  
  # if predlabel==classes:
  #   print('equal')
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. 

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  # print(rnd,data)
  split1 = rnd<percent
  # print(type(data),type(split1))
  # print(split1)
  # print('hello')
  # print(data[split1,:])
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

In [ ]:
def divide(data,label,perce):
  d=rng.random(len(label))
  d=rng.integers(low=0,high=max(label),size=len(label))
  d=d>2

  return data[d,:]

In [ ]:
s=divide(dataset.data,dataset.target,0.8)

In [ ]:
len(s)

8314

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:

testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4162
Number of other samples =  16478
Percent of test data =  20.164728682170544 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

In [ ]:
traindata

array([[   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       [   5.6431    ,   52.        ,    5.8173516 , ...,    2.54794521,
          37.85      , -122.25      ],
       [   3.8462    ,   52.        ,    6.28185328, ...,    2.18146718,
          37.85      , -122.25      ],
       ...,
       [   1.5603    ,   25.        ,    5.04545455, ...,    2.56060606,
          39.48      , -121.09      ],
       [   2.5568    ,   18.        ,    6.11403509, ...,    3.12280702,
          39.49      , -121.21      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]])

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
# import pandas as pd
# pd.Series(trainpred).unique()
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.16772100567721004


In [ ]:
len(trainpred)

12330

For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34956605593056894
Validation accuracy using random classifier is  0.1711668273866924


In [ ]:
valpred

array([2, 4, 0, ..., 5, 5, 1])

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. 

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.3308714918759232


You can run the above cell multiple times to try with different random splits. 
We notice that the accuracy is different for each run, but close together. 

Now let us compare it with the accuracy we get on the test dataset. 

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34406535319557907


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced? 

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

In [ ]:
import pandas as pd
l,p=[],[]
for x in range(10,100,10):
  l=[]
  traindata , trainlabel , valdata ,vallabel = split(alltraindata,alltrainlabel,x/100)
  prednn = NN(traindata,trainlabel,valdata)
  accu = Accuracy(vallabel,prednn)
  l.append(100-x)
  l.append(accu)
  p.append(l)
pd.DataFrame(p,index=list(range(9)),columns=['% of validation ','Accuracy of Nearest Neighbour'])

,% of validation,Accuracy of Nearest Neighbour
0,90,0.299879
1,80,0.306790
2,70,0.317624
3,60,0.324066
4,50,0.326536
5,40,0.332659
6,30,0.335294
7,20,0.339691
8,10,0.343301


In [ ]:
import pandas as pd
l,p=[],[]
for x in range(10,100,10):
  l=[]
  traindata , trainlabel , valdata ,vallabel = split(alltraindata,alltrainlabel,x/100)
  prednn = RandomClassifier(traindata,trainlabel,valdata)
  accu = Accuracy(vallabel,prednn)
  l.append(100-x)
  l.append(accu)
  p.append(l)
pd.DataFrame(p,index=list(range(9)),columns=['% of validation ','Accuracy in Random Classifer'])

,% of validation,Accuracy in Random Classifer
0,90,0.166363
1,80,0.163465
2,70,0.164257
3,60,0.164480
4,50,0.168164
5,40,0.169440
6,30,0.170560
7,20,0.159864
8,10,0.167166


**Answer to Q)1: we can see from above table that if we increase the percentage of validation set then accuracy decreases in nearest neighbour but in the random classifier the Accuracy remains same and if we reduce the validation set than the accuracy will increase for Nearest neighbour and remains constant for Random Classifier**

In [ ]:
import pandas as pd
l,p=[],[]
for x in range(10,100,10):
  l=[]
  traindata , trainlabel , valdata ,vallabel = split(alltraindata,alltrainlabel,x/100)
  prednn = NN(traindata,trainlabel,valdata)
  accu = Accuracy(vallabel,prednn)
  testpredn = NN(traindata,trainlabel,testdata)
  test_accuracy = Accuracy(testlabel,testpredn)
  l.append(100-x)
  l.append(x)
  l.append(test_accuracy)
  p.append(l)
pd.DataFrame(p,index=list(range(9)),columns=['% of validation ','% train','Test_accuracy'])


,% of validation,% train,Test_accuracy
0,90,10,0.295673
1,80,20,0.311298
2,70,30,0.317067
3,60,40,0.328365
4,50,50,0.331971
5,40,60,0.337019
6,30,70,0.344952
7,20,80,0.344231
8,10,90,0.350240


**Answer for Q)2: From the above table we can see that training data should be as high as possible and validation should be as low as possible.But if we make validation data as zero for high Accuracy then we cannot tune the Hyper parameters here in our case we donot have hyper parameters if there of tuning then we can get better Accuracy value than the above Value.**

**Answer for Q)3:  It will be always good to have 80-90% for traing data and 20-10% for Validationdata from Total training data**

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. 

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.34285883639454606
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


**Answer for Q)1:** we can see that validation accuracy is increased and have consistent value of accuracy using the multiple splits

**Answer for Q)2:**No it doesnot give higher than test accuracy value but the vlaidation accuracy and test accuracy are nearly same

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 2, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33905246039708675
test accuracy is  0.34917953667953666


In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 3, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.3370479225709515
test accuracy is  0.34917953667953666


In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 4, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.34349593911857196
test accuracy is  0.34917953667953666


In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 20, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.3415042943882442
test accuracy is  0.34917953667953666


**Answer for Q)3:** we can see the validation accuracy increases upto certain iteration and if we increase further more the validation accuracy remains almost same even we increase the iterations to 20



**Answe for Q)4:** If we have very less training data then even if we increase the number of iterations the accuracy will remians constant and it won't increase further.If we want high accuracy on test data then we **cannot** deal with very small training or validation dataset unless we have done some data agumentation